In [1]:
import numpy as np
import json
import tensorflow as tf


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 5GB of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)
 

file = open("tweets_DM.json", 'r', encoding='utf-8')
tweet = []
for line in file.readlines():
    t = json.loads(line)
    tweet.append(t)

print(len(tweet))



1 Physical GPUs, 1 Logical GPUs
1867535


In [2]:
emo = (open("emotion.csv", 'r', encoding='utf-8'))
emot = {}

for x in emo:
    s = x.split(',')
    emot[s[0]] = s[1].split('\n')[0] 
    
print(emot['0x336431'])


trust


In [3]:
data = (open("data_identification.csv", 'r', encoding='utf-8'))
train = {}
test = {}
for x in data:
    s = x.split(',')
    if s[1] == 'train\n':
        train[s[0]] = s[1]
    if s[1] == 'test\n':
        test[s[0]] = s[1] 
print('0x336431' in train)
print('0x336431' in test)

True
False


In [4]:
print(tweet[0])

{'_score': 391, '_index': 'hashtag_tweets', '_source': {'tweet': {'hashtags': ['Snapchat'], 'tweet_id': '0x376b20', 'text': 'People who post "add me on #Snapchat" must be dehydrated. Cuz man.... that\'s <LH>'}}, '_crawldate': '2015-05-23 11:42:47', '_type': 'tweets'}


In [4]:
import pandas as pd

X_train = []
Y_train = []
X_test = []
ylabel = []

for x in tweet:
    if x['_source']['tweet']['tweet_id'] in train:
        X_train.append({'score':x['_score'],'text':x['_source']['tweet']['text']})
        Y_train.append({'emotion':emot[x['_source']['tweet']['tweet_id']]})
    else:
        X_test.append({'score':x['_score'],'text':x['_source']['tweet']['text']})
        ylabel.append(x['_source']['tweet']['tweet_id'])


    

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
print(len(X_train))
print(len(Y_train))
print(len(X_test))
print(len(ylabel))

tweet = []
xtrain = pd.DataFrame(X_train)
ytrain = pd.DataFrame(Y_train)
xtest = pd.DataFrame(X_test)
ytrain = ytrain.to_numpy()

BOW_500 = CountVectorizer(max_features=500, tokenizer=nltk.word_tokenize) 

BOW_500.fit(xtrain['text'])
xtrain = BOW_500.transform(xtrain['text'])
xtest = BOW_500.transform(xtest['text'])

1455563
1455563
411972
411972


C:\Users\james\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [8]:
from sklearn.preprocessing import LabelEncoder
import keras

ytrain = pd.DataFrame(Y_train)

label_encoder = LabelEncoder()
label_encoder.fit(ytrain)
enc = label_encoder.transform(ytrain)
ytrain = keras.utils.to_categorical(enc)


C:\Users\james\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [10]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *


 

def DNN(input_size=(500,)):
    inputs = Input(input_size)
    layer1 = Dense(units = 128, activation = 'relu')(inputs)
    layer2 = Dense(units = 64, activation = 'relu')(layer1)
    layer3 = Dense(units = 64, activation = 'relu')(layer2)
    layer4 = Dense(units = 64, activation = 'relu')(layer3)
    layer5 = Dense(units = 64, activation = 'relu')(layer4)
    output = Dense(units = 8, activation = 'softmax')(layer5)
    
    model = Model(inputs = inputs, outputs = output)
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    model._name = 'DNN'
    model.summary()
    
    return model


print(type(xtrain))
print(type(ytrain))
model = DNN()
history = model.fit(xtrain[:1300000], ytrain[:1300000], 
                    epochs=10, 
                    batch_size=32,
                    validation_data = (xtrain[1300000:], ytrain[1300000:]),
                    verbose = 1)

<class 'scipy.sparse.csr.csr_matrix'>
<class 'numpy.ndarray'>
Model: "DNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 500)]             0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               64128     
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_8 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_9 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_10 (Dense)             (None, 64)                4160      
_________________________________________________________________
d

In [11]:
result = model.predict(xtest)
print(result[0])
dec = np.argmax(result, axis=1)
predict = label_encoder.inverse_transform(dec)
print(predict[0])

[0.0008998  0.7112919  0.00167445 0.00253036 0.18896344 0.0055354
 0.00236254 0.08674213]
anticipation


In [14]:
import csv

file = open("submission_example.csv",'w',encoding = 'utf-8',newline = '')
f = csv.writer(file)
f.writerow(['id', 'emotion'])
for i in range(len(predict)):
    f.writerow([ylabel[i],predict[i]])
file.close()